<a href="https://colab.research.google.com/github/ShayanPervez/Advanced-RAG/blob/main/hybrid_search_and_reranking_in_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U langchain langchain-community weaviate-client==3.*

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.1/120.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00


In [2]:
import weaviate


In [3]:
WEAVIATE_CLUSTER = "Enter the URL"
WEAVIATE_API_KEY ="Enter your API key"

In [4]:
WEAVIATE_URL = WEAVIATE_CLUSTER

In [5]:
HF_TOKEN = "hf_xxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

In [6]:
client = weaviate.Client(
    url=WEAVIATE_URL,
    auth_client_secret= weaviate.AuthApiKey(WEAVIATE_API_KEY),
    additional_headers={
        "X-HuggingFace-Api-Key": HF_TOKEN,
    }
)

In [7]:
client.is_ready()

True

In [8]:
client.schema.get()

{'classes': [{'class': 'RAG',
   'description': 'Documents for RAG',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None},
    'usingBlockMaxWAND': True},
   'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2',
     'type': 'sentence',
     'useCache': True,
     'useGPU': False,
     'vectorizeClassName': True,
     'waitForModel': False}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the paragraph',
     'indexFilterable': True,
     'indexRangeFilters': False,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-huggingface': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "T

In [9]:
from os import name
schema = {
  "classes": [
    {
      "class": "RAG",
      "description": "Documents for RAG",
      "vectorizer": "text2vec-huggingface",
      "moduleConfig": {
        "text2vec-huggingface": {
          "model": "sentence-transformers/all-MiniLM-L6-v2",
          "type": "sentence"
        }
      },
      "properties": [
        {
          "dataType": ["text"],
          "description": "The content of the paragraph",
          "moduleConfig": {
            "text2vec-huggingface": {
              "skip": False,
              "vectorizePropertyName": False
            }
          },
          "name":"content",
        },
      ],
    },
  ]
}


In [10]:
#client.schema.create(schema)

In [11]:
client.schema.get()

{'classes': [{'class': 'RAG',
   'description': 'Documents for RAG',
   'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2},
    'cleanupIntervalSeconds': 60,
    'stopwords': {'additions': None, 'preset': 'en', 'removals': None},
    'usingBlockMaxWAND': True},
   'moduleConfig': {'text2vec-huggingface': {'model': 'sentence-transformers/all-MiniLM-L6-v2',
     'type': 'sentence',
     'useCache': True,
     'useGPU': False,
     'vectorizeClassName': True,
     'waitForModel': False}},
   'multiTenancyConfig': {'autoTenantActivation': False,
    'autoTenantCreation': False,
    'enabled': False},
   'properties': [{'dataType': ['text'],
     'description': 'The content of the paragraph',
     'indexFilterable': True,
     'indexRangeFilters': False,
     'indexSearchable': True,
     'moduleConfig': {'text2vec-huggingface': {'skip': False,
       'vectorizePropertyName': False}},
     'name': 'content',
     'tokenization': 'word'},
    {'dataType': ['text'],
     'description': "T

In [12]:
from langchain.retrievers.weaviate_hybrid_search import WeaviateHybridSearchRetriever

In [13]:
retriever = WeaviateHybridSearchRetriever(
    client =client,
    index_name= "RAG",
    text_key="content",
    alpha=0.5,
    attributes=[],
    create_schema_if_missing=True
)

/tmp/ipython-input-13-532022860.py:1: LangChainDeprecationWarning: The class `WeaviateHybridSearchRetriever` was deprecated in LangChain 0.3.18 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-weaviate package and should be used instead. To use it run `pip install -U :class:`~langchain-weaviate` and import as `from :class:`~langchain_weaviate import WeaviateVectorStore``.
  retriever = WeaviateHybridSearchRetriever(


In [14]:
model_name = "google/gemma-2b-it"

In [15]:
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [16]:
pip install "transformers>=4.45.1"

In [66]:
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig)
from langchain import HuggingFacePipeline

In [48]:
def load_quantized_model(model_name:str):
  bnb_config = BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_use_double_quant=True,
      bnb_4bit_quant_type="nf4",
      bnb_4bit_compute_dtype=torch.bfloat16,
      low_cpu_mem_usage=True,
  )

  model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config=bnb_config,
      device_map="auto",
      torch_dtype=torch.bfloat16,
  )
  return model

In [49]:
def initalize_tokenizer(model_name:str):
  tokenizer = AutoTokenizer.from_pretrained(model_name,return_token_type_ids=False)
  tokenizer.bos_token_id = 1
  return tokenizer

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [22]:
tokenizer = initalize_tokenizer(model_name)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [45]:
model = load_quantized_model(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [67]:
pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=5000,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id
)

Device set to use cuda:0


In [68]:
llm = HuggingFacePipeline(pipeline=pipeline)

In [52]:
doc_path = "/content/2005.11401v4.pdf"

In [53]:
!pip install pypdf

In [54]:
from langchain_community.document_loaders import PyPDFLoader

In [55]:
loader = PyPDFLoader(doc_path)

In [56]:
docs = loader.load()

In [57]:
docs

[Document(metadata={'producer': 'pdfTeX-1.40.21', 'creator': 'LaTeX with hyperref', 'creationdate': '2021-04-13T00:48:38+00:00', 'author': '', 'keywords': '', 'moddate': '2021-04-13T00:48:38+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.14159265-2.6-1.40.21 (TeX Live 2020) kpathsea version 6.3.2', 'subject': '', 'title': '', 'trapped': '/False', 'source': '/content/2005.11401v4.pdf', 'total_pages': 19, 'page': 0, 'page_label': '1'}, page_content='Retrieval-Augmented Generation for\nKnowledge-Intensive NLP Tasks\nPatrick Lewis†‡, Ethan Perez⋆,\nAleksandra Piktus†, Fabio Petroni†, Vladimir Karpukhin†, Naman Goyal†, Heinrich Küttler†,\nMike Lewis†, Wen-tau Yih†, Tim Rocktäschel†‡, Sebastian Riedel†‡, Douwe Kiela†\n†Facebook AI Research; ‡University College London; ⋆New York University;\nplewis@fb.com\nAbstract\nLarge pre-trained language models have been shown to store factual knowledge\nin their parameters, and achieve state-of-the-art results when ﬁne-tuned on down-\nstream NLP 

In [58]:
len(docs)

19

In [99]:
def clean_doc_metadata(docs):
    cleaned_docs = []
    for doc in docs:
        new_metadata = {}
        for key, value in doc.metadata.items():
            new_key = key.replace('.', '_')  # Replace '.' with '_'
            new_metadata[new_key] = value
        doc.metadata = new_metadata
        cleaned_docs.append(doc)
    return cleaned_docs

cleaned_docs = clean_doc_metadata(docs)
retriever.add_documents(cleaned_docs)


[ERROR] Batch ConnectionError Exception occurred! Retrying in 2s. [1/3]


['da849de5-dab0-413e-b1d1-2138940170b2',
 'ae5e485e-6537-4611-a66f-0a168e389d02',
 '61ed2ed6-43f1-4e94-bab0-63a441b4516f',
 '65d65af2-1995-4f18-bfe1-53371b4039a7',
 '948cffe4-88f8-40ce-b76c-6bf83959d541',
 '54a567de-2178-45fc-b2bf-15ec80bfbc51',
 'b8bb3609-4d88-4248-9605-7724393071d4',
 '30df8377-74e2-4e32-8262-23d5382f1891',
 '131c79b2-5339-4d63-a796-22f36fdb59ce',
 '57de5203-87a7-426e-ac0b-41241065490d',
 'd85ebb9c-53d9-47e9-9c85-5b229469bca9',
 '9d3c400e-4094-491c-9d08-39b1295f01b2',
 '9dfeeec3-49be-45b8-81f2-3ae7d771e3ca',
 '51cd3441-4405-409f-a287-9de616d79215',
 '50fa38ce-7dad-48eb-9e74-5a4b601931df',
 '67e94257-656d-48cd-af40-acbc21acf3c2',
 '2872dd2c-8c16-41c1-8b68-09e0dd3afe00',
 'cc093edb-cf7c-422d-bca9-475e31bd0c2e',
 'ddaa3fb7-d23d-4427-8852-4b35fb909d4b']

In [100]:
retriever.invoke("What is RAG token?")

[Document(metadata={}, page_content='Document 1: his works are considered classics of American\nliterature ... His wartime experiences formed the basis for his novel\n”A Farewell to Arms”(1929) ...\nDocument 2: ... artists of the 1920s ”Lost Generation” expatriate\ncommunity. His debut novel,”The Sun Also Rises”, was published\nin 1926.\nBOS\n”\nTheSunAlso\nR ises\n” is a\nnovel\nby this\nauthor\nof ” A\nFarewellto\nArms\n”\nDoc 1\nDoc 2\nDoc 3\nDoc 4\nDoc 5\nFigure 2: RAG-Token document posterior p(zi|x,yi,y−i) for each generated token for input “Hem-\ningway" for Jeopardy generation with 5 retrieved documents. The posterior for document 1 is high\nwhen generating “A Farewell to Arms" and for document 2 when generating “The Sun Also Rises".\nTable 3: Examples from generation tasks. RAG models generate more speciﬁc and factually accurate\nresponses. ‘?’ indicates factually incorrect responses, * indicates partially correct responses.\nTask Input Model Generation\nMS-\nMARCO\ndeﬁne midd

In [101]:
len(retriever.invoke("What is RAG token?"))

4

In [102]:
retriever.invoke("What is RAG token?", score=True)

[Document(metadata={'_additional': {'explainScore': '\nHybrid (Result Set keyword,bm25) Document d453eef6-7d4b-4170-a247-4c0297662efc: original score 1.691593, normalized score: 0.43592343 - \nHybrid (Result Set vector,hybridVector) Document d453eef6-7d4b-4170-a247-4c0297662efc: original score 0.422423, normalized score: 0.5', 'score': '0.93592346'}}, page_content='Document 1: his works are considered classics of American\nliterature ... His wartime experiences formed the basis for his novel\n”A Farewell to Arms”(1929) ...\nDocument 2: ... artists of the 1920s ”Lost Generation” expatriate\ncommunity. His debut novel,”The Sun Also Rises”, was published\nin 1926.\nBOS\n”\nTheSunAlso\nR ises\n” is a\nnovel\nby this\nauthor\nof ” A\nFarewellto\nArms\n”\nDoc 1\nDoc 2\nDoc 3\nDoc 4\nDoc 5\nFigure 2: RAG-Token document posterior p(zi|x,yi,y−i) for each generated token for input “Hem-\ningway" for Jeopardy generation with 5 retrieved documents. The posterior for document 1 is high\nwhen genera

In [103]:
from langchain.chains import RetrievalQA

In [104]:
hybrid_chain = RetrievalQA.from_chain_type(llm = llm, chain_type="stuff", retriever=retriever)

In [105]:
result = hybrid_chain.invoke("What is RAG token?")

In [106]:
print(result)

{'query': 'What is RAG token?', 'result': 'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nDocument 1: his works are considered classics of American\nliterature ... His wartime experiences formed the basis for his novel\n”A Farewell to Arms”(1929) ...\nDocument 2: ... artists of the 1920s ”Lost Generation” expatriate\ncommunity. His debut novel,”The Sun Also Rises”, was published\nin 1926.\nBOS\n”\nTheSunAlso\nR ises\n” is a\nnovel\nby this\nauthor\nof ” A\nFarewellto\nArms\n”\nDoc 1\nDoc 2\nDoc 3\nDoc 4\nDoc 5\nFigure 2: RAG-Token document posterior p(zi|x,yi,y−i) for each generated token for input “Hem-\ningway" for Jeopardy generation with 5 retrieved documents. The posterior for document 1 is high\nwhen generating “A Farewell to Arms" and for document 2 when generating “The Sun Also Rises".\nTable 3: Examples from generation tasks. RAG models generate more speciﬁ

In [107]:
result2 = hybrid_chain.invoke("What is RAG token")

In [108]:
result3 = hybrid_chain.invoke("What is RAG token?")

In [109]:
COHERE_API_KEY = "1llO4b7W4W4bGRN4OMmzLg3pQfutG3AsndJGuEGa"

In [110]:
!pip install cohere

In [116]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import CohereRerank

In [120]:
compressor = CohereRerank(cohere_api_key=COHERE_API_KEY, model="rerank-english-v3.0")

In [121]:
compression_retriver = ContextualCompressionRetriever(base_compressor=compressor,base_retriever=retriever)

In [123]:
compressed_doc = compression_retriver.get_relevant_documents("What is RAG token?")

In [127]:
print(compressed_doc)

[Document(metadata={'relevance_score': 0.9890131}, page_content='by θthat generates a current token based on a context of the previous i−1 tokens y1:i−1, the original\ninput xand a retrieved passage z.\nTo train the retriever and generator end-to-end, we treat the retrieved document as a latent variable.\nWe propose two models that marginalize over the latent documents in different ways to produce a\ndistribution over generated text. In one approach, RAG-Sequence, the model uses the same document\nto predict each target token. The second approach, RAG-Token, can predict each target token based\non a different document. In the following, we formally introduce both models and then describe the\npη and pθ components, as well as the training and decoding procedure.\n2.1 Models\nRAG-Sequence Model The RAG-Sequence model uses the same retrieved document to generate\nthe complete sequence. Technically, it treats the retrieved document as a single latent variable that\nis marginalized to get t

In [128]:
hybrid_chain = RetrievalQA.from_chain_type(llm = llm, chain_type="stuff", retriever=compression_retriver)

In [129]:
response = hybrid_chain.invoke("What is RAG token?")

In [130]:
print(response.get("result"))

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

by θthat generates a current token based on a context of the previous i−1 tokens y1:i−1, the original
input xand a retrieved passage z.
To train the retriever and generator end-to-end, we treat the retrieved document as a latent variable.
We propose two models that marginalize over the latent documents in different ways to produce a
distribution over generated text. In one approach, RAG-Sequence, the model uses the same document
to predict each target token. The second approach, RAG-Token, can predict each target token based
on a different document. In the following, we formally introduce both models and then describe the
pη and pθ components, as well as the training and decoding procedure.
2.1 Models
RAG-Sequence Model The RAG-Sequence model uses the same retrieved document to generate
the complete sequence. Technically, it